In [ ]:
from astropy.table import Table, vstack
from astropy.io import fits
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
from matplotlib.patches import Circle
import pandas as pd
from datetime import datetime
import sys
size_outside = 400 # phe
import copy
start_date_2020 = datetime.strptime("2019-11-25 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2020 = datetime.strptime("2020-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2021 = datetime.strptime("2021-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2021 = datetime.strptime("2021-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2022 = datetime.strptime("2022-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2022 = datetime.strptime("2022-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2023 = datetime.strptime("2023-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2023 = datetime.strptime("2023-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

start_date_2024 = datetime.strptime("2024-01-01 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()
end_date_2024 = datetime.strptime("2024-12-31 00:00:00.0", "%Y-%m-%d %H:%M:%S.%f").timestamp()

ring_size_upper = 10000
ring_completeness_cut = 0.9
ring_containment_cut = 0.9

sys.path.append("/Users/vdk/Software/code")
from python_modules.muonpipe.usefull_func import hist_wo_outliers


In [ ]:
muon_files = glob.glob('/Users/vdk/muons2024/all_muon_fits/low_nsb_filter/*')
print(sorted(muon_files))

df_files = [pd.read_csv(muon_file, na_values=['NA', '?']) for muon_file in sorted(muon_files)[3:]] 
df_all_data = pd.concat(df_files, ignore_index=True)
df_good_data_real = df_all_data[(df_all_data['muon_efficiency'] < 1) & (df_all_data['size_outside'] < 500)]
del(df_all_data)
df_good_data_real

In [ ]:
listdir_perf_paper = glob.glob("/Users/vdk/muons2024/lapalma_simulations/performance_paper/tables/*")

listdir_global_peak = glob.glob("/Users/vdk/muons2024/lapalma_simulations/global_peak/tables/*")

dat = Table.read('{}'.format(listdir_perf_paper[0]), format='fits')

for muon_file in listdir_perf_paper:
    dat2 = Table.read(muon_file, format='fits')
    dat2['good_ring'] = dat2['good_ring'].astype(bool)
    dat2['is_valid'] = dat2['is_valid'].astype(bool)
    dat2['parameters_at_limit'] = dat2['parameters_at_limit'].astype(bool)
    dat = vstack([dat, dat2])
min_impact = min(df_good_data_real['impact_parameter'])
max_impact = max(df_good_data_real['impact_parameter']) 
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500) &
                      (df['impact_parameter'] < max_impact) &
                      (df['impact_parameter'] > min_impact)]


dat = Table.read('{}'.format(listdir_global_peak[0]), format='fits')
for muon_file in listdir_global_peak:
    dat2 = Table.read(muon_file, format='fits')
    dat2['good_ring'] = dat2['good_ring'].astype(bool)
    dat = vstack([dat, dat2])
min_impact = min(df_good_data_real['impact_parameter'])
max_impact = max(df_good_data_real['impact_parameter']) 
df = dat.to_pandas()
df_good_data_global = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500) &
                      (df['impact_parameter'] < max_impact) &
                      (df['impact_parameter'] > min_impact)]

In [ ]:
ring_completeness_cut = 0.9
ring_containment_cut = 0.9
bin_number = 8


colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#e377c2',  # raspberry yogurt pink
    '#8c564b',  # chestnut brown
]

alpha_reg = 1

plt.figure(figsize = (7,7))

flag = '2023-24'


sns.regplot(x = df_good_data_real['ring_radius'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) & 
                                                 (df_good_data_real['ring_containment'] > ring_containment_cut) & 
                                                 (df_good_data_real['size_outside'] < size_outside) &  
                                                 (df_good_data_real['ring_size'] < ring_size_upper) &
                                                (df_good_data_real['event_time'] > start_date_2023) &
                                                (df_good_data_real['event_time'] < end_date_2023)], 
            y = df_good_data_real['ring_size'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) & 
                                               (df_good_data_real['ring_containment'] > ring_containment_cut) & 
                                               (df_good_data_real['size_outside'] < size_outside) & 
                                               (df_good_data_real['ring_size'] < ring_size_upper) & 
                                                (df_good_data_real['event_time'] > start_date_2023) &
                                                (df_good_data_real['event_time'] < end_date_2023)], 
            x_bins=bin_number, color='red', label = 'Data for 2023 year',scatter_kws={'alpha':alpha_reg}, line_kws={'alpha': alpha_reg})

sns.regplot(x = df_good_data_real['ring_radius'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) & 
                                                 (df_good_data_real['ring_containment'] > ring_containment_cut) & 
                                                 (df_good_data_real['size_outside'] < size_outside) &  
                                                 (df_good_data_real['ring_size'] < ring_size_upper) &
                                                (df_good_data_real['event_time'] > start_date_2024) &
                                                (df_good_data_real['event_time'] < end_date_2024)], 
            y = df_good_data_real['ring_size'][(df_good_data_real['ring_completeness'] > ring_completeness_cut) & 
                                               (df_good_data_real['ring_containment'] > ring_containment_cut) & 
                                               (df_good_data_real['size_outside'] < size_outside) & 
                                               (df_good_data_real['ring_size'] < ring_size_upper) & 
                                                (df_good_data_real['event_time'] > start_date_2024) &
                                                (df_good_data_real['event_time'] < end_date_2024)], 
            x_bins=bin_number, color='orange', label = 'Data for 2024 year',scatter_kws={'alpha':alpha_reg}, line_kws={'alpha': alpha_reg})


sns.regplot(x = df_good_data['ring_radius'][(df_good_data['ring_completeness'] > 0.9) &
                                            (df_good_data['ring_containment'] > ring_containment_cut) &  
                                            (df_good_data['size_outside'] < size_outside) & 
                                            (df_good_data['ring_size'] < ring_size_upper) & 
                                            (df_good_data['muon_efficiency'] < 1)], 
            y = df_good_data['ring_size'][(df_good_data['ring_completeness'] > 0.9) & 
                                          (df_good_data['ring_containment'] > ring_containment_cut) & 
                                          (df_good_data['size_outside'] < size_outside) & 
                                          (df_good_data['ring_size'] < ring_size_upper) & 
                                          (df_good_data['muon_efficiency'] < 1)], 
            x_bins=bin_number, color='k', label = 'Simulations')

sns.regplot(x = df_good_data_global['ring_radius'][(df_good_data_global['ring_completeness'] > 0.9) &
                                            (df_good_data_global['ring_containment'] > ring_containment_cut) &  
                                            (df_good_data_global['size_outside'] < size_outside) & 
                                            (df_good_data_global['ring_size'] < ring_size_upper) & 
                                            (df_good_data_global['muon_efficiency'] < 1)], 
            y = df_good_data_global['ring_size'][(df_good_data_global['ring_completeness'] > 0.9) & 
                                          (df_good_data_global['ring_containment'] > ring_containment_cut) & 
                                          (df_good_data_global['size_outside'] < size_outside) & 
                                          (df_good_data_global['ring_size'] < ring_size_upper) & 
                                          (df_good_data_global['muon_efficiency'] < 1)], 
            x_bins=bin_number, color='k', label = 'Simulations with GlobalPeakWindowSum')




plt.xlabel('Muon radius [deg]')
plt.ylabel('Muon Size [pe]')
plt.grid(linestyle="dashed")
plt.ylim(2000,3500)
plt.xlim(0.95,1.25)

plt.legend(prop={'size': 12}, loc=2)
#plt.show()
#plt.savefig(f'/Users/vdk/size_{flag}_nsb15.png', dpi=200)
#plt.savefig(f'/Users/vdk/width_{flag}_n.png', dpi=200)

In [ ]:
df_good_data_real.shape

In [ ]:
df_good_data_global['ring_size'][(df_good_data_global['ring_completeness'] > 0.9) & 
                                          (df_good_data_global['ring_containment'] > ring_containment_cut) & 
                                          (df_good_data_global['size_outside'] < size_outside) & 
                                          (df_good_data_global['ring_size'] < ring_size_upper) & 
                                          (df_good_data_global['muon_efficiency'] < 1)]